In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
import ast


In [58]:
sample_sub = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()


,id,context,question,answers,label
0,train_0,"In rheumatoid arthritis, the body' s immune sy...","The statements above, if true, most strongly s...",['Unlike aspirin and other medications that re...,1
1,train_1,Patient: Pharmacists maintain that doctors sho...,The patient's argument proceeds by,['attempting to discredit a position by questi...,0
2,train_2,Paula will visit the dentist tomorrow morning ...,The pattern of reasoning displayed above most ...,"['If Marge goes to the bank today, Lauren will...",1
3,train_3,Some theorists argue that literary critics sho...,The argument's conclusion follows logically if...,"[""Any critic who is able to help readers make ...",1
4,train_4,Shipping Clerk: The five specially ordered shi...,"If the shipping clerk's statements are true, w...",['At least one of the shipments sent to Truax ...,0


In [59]:
# Splitting up the answers into a list of answers
train['answers'] = train['answers'].apply(ast.literal_eval)

# Creating a column for the correct answer based on the label
train['correct_answer'] = train.apply(lambda row: row['answers'][row['label']], axis=1)


In [60]:
train.head()

,id,context,question,answers,label,correct_answer
0,train_0,"In rheumatoid arthritis, the body' s immune sy...","The statements above, if true, most strongly s...",[Unlike aspirin and other medications that red...,1,A patient treated with the new medication for ...
1,train_1,Patient: Pharmacists maintain that doctors sho...,The patient's argument proceeds by,[attempting to discredit a position by questio...,0,attempting to discredit a position by question...
2,train_2,Paula will visit the dentist tomorrow morning ...,The pattern of reasoning displayed above most ...,"[If Marge goes to the bank today, Lauren will ...",1,Kevin will wash his car tomorrow only if Britt...
3,train_3,Some theorists argue that literary critics sho...,The argument's conclusion follows logically if...,[Any critic who is able to help readers make t...,1,If it is impossible to produce completely valu...
4,train_4,Shipping Clerk: The five specially ordered shi...,"If the shipping clerk's statements are true, w...",[At least one of the shipments sent to Truax C...,0,At least one of the shipments sent to Truax Co...


In [11]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# Prepare your dataset (context, question, answer, label)
dataset = [i for i in train.values] # This should be a list of (input, label) pairs
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Fine-tuning the model
for epoch in range(10):
    for batch in train_loader:
        inputs, labels = batch
        model.train(inputs, labels)

# Save and evaluate your model
model.save_pretrained('./my_finetuned_model')
evaluate_model(model, test_dataset)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object